<a href="https://colab.research.google.com/github/antoniovfonseca/agentic-ai-global-lulc/blob/main/notebooks/overview-glance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/antoniovfonseca/agentic-ai-global-lulc/blob/main/notebooks/overview-glance.ipynb)

## **1. Import of geospatial processing and visualization modules.**
---



In [5]:
# Enter your google cloud project id
project_id = "ee-antoniovicgf"

In [6]:
import os
import sys
import warnings

# Filter future warnings to keep output clean
warnings.simplefilter(action='ignore', category=FutureWarning)

# Install geemap dependency
print("Installing dependencies...")
!pip install geemap -q

# Define repository details
repo_url = 'https://github.com/antoniovfonseca/agentic-ai-global-lulc.git'
repo_name = 'agentic-ai-global-lulc'

# Clone or update the repository
if not os.path.exists(repo_name):
    print(f"Cloning {repo_name}...")
    !git clone {repo_url}
else:
    print(f"Updating {repo_name}...")
    !cd {repo_name} && git pull

# Add the notebooks directory to system path to allow imports
notebooks_path = os.path.join(repo_name, 'notebooks')
if notebooks_path not in sys.path:
    sys.path.append(os.path.abspath(notebooks_path))
    print(f"Added {notebooks_path} to system path.")

Installing dependencies...
Updating agentic-ai-global-lulc...
Already up to date.
Added agentic-ai-global-lulc/notebooks to system path.


If you are running this notebook in Google Colab, please uncomment and run the following cell to install the required packages.

## **2. Earth Engine session authentication and initialization.**


---
See the Google instructions [here](https://developers.google.com/earth-engine/guides/auth).


In [7]:
import ee

# Authenticate and initialize earth engine
try:
    # Try initializing with the provided project id or default
    if project_id:
        ee.Initialize(project=project_id)
    else:
        ee.Initialize()
    print("Earth Engine initialized successfully.")

except Exception:
    # Trigger authentication flow if initialization fails
    print("Authentication required...")
    ee.Authenticate()

    try:
        # Retry initialization after authentication
        if project_id:
            ee.Initialize(project=project_id)
        else:
            ee.Initialize()
        print("Earth Engine initialized successfully.")

    except ee.EEException:
        # Fallback to manual input if no project is found
        print("No default project found.")
        manual_id = input("Please enter your Google Cloud Project ID: ")
        ee.Initialize(project=manual_id)
        print("Earth Engine initialized successfully.")

Earth Engine initialized successfully.


In [8]:
try:
    # Import the visualization function from utils
    from utils import get_glance_map
    print("Success: get_glance_map function imported.")
except ImportError as e:
    print(f"Error: Could not import utils. Details: {e}")

Success: get_glance_map function imported.


## **3. Definition of the remote GLANCE dataset source.**

In [9]:
# 2. Import the custom function from the utils file
from utils import get_glance_map

# 3. Define parameters
target_year = 2015

# 4. Run the function imported from utils.py
glance_map = get_glance_map(target_year)

# 5. Display the map
if glance_map:
    display(glance_map)

GLANCE map for year 2015 generated successfully.


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…